## 2. Classification linéaire : le Perceptron


**2.1 Classification de points du plan avec un perceptron**

_**Rappel du cours :**_
Le perceptron est une architecture simple de réseau de neurones utilisée pour la classification binaire. Le modèle linéaire associé est défini par :

$$
\hat{y} = \sigma(Wx + b)
$$

où $W$ est la matrice de poids, $b$ est le vecteur de biais, $x$ est l'entrée et $\sigma$ est la fonction d'activation.

La fonction d'activation la plus courante pour le perceptron est la fonction signe :

$$
\sigma(z) = \begin{cases}
      1 & z \geq 0 \\
      -1 & z < 0
   \end{cases}
$$

Lors de l'apprentissage, on met à jour les poids et les biais avec la règle suivante :

$$
W \leftarrow W + \eta (y - \hat{y})x \\
b \leftarrow b + \eta (y - \hat{y})
$$

où $\eta$ est le taux d'apprentissage.


### 2.1   classification points du plan


Générez un ensemble de données de points en deux dimensions avec deux classes séparables linéairement. Implémentez un perceptron qui apprend à classer ces points en utilisant la fonction signe comme fonction d'activation.

**Question 1.** Générez les données : créez deux groupes de points avec des coordonnées $(x,y)$ uniformément réparties, chacun étant associé à une classe différente.

``` python
import numpy as np

# Nombre de points à générer
n_points = 100


# Génération des données en prenant deux distribution normales
class_1 = np.random.normal(-3, 1, size=(n_points, 2))
class_2 = np.random.normal(3 , 1, size=(n_points, 2))
```



**Question 2 :**  Affichez les points avec des marqueurs différents pour chaque classe.

``` python
import matplotlib.pyplot as plt

# Affichage des points
plt.scatter(class_1[:, 0], class_1[:, 1], marker='x', color='blue', label='Class 1')
plt.scatter(class_2[:, 0], class_2[:, 1], marker='o', color='red', label='Class 2')

plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.legend()
```



**Question 3:** Implémentez un perceptron pour classer ces points. Initialisez les poids et le biais avec des valeurs nulles.

``` python
# Fonction d'activation signe
def sign_activation(z):
    return 1 if z >= 0 else -1

# Initialisation des poids et du biais
weights = np.zeros(2)
bias = 0
learning_rate = 0.1
```



**Question 4 :** Faites une passe d'apprentissage sur l'ensemble des données. Mettez à jour les poids et le biais selon la règle d'apprentissage du perceptron.

``` python
for point, label in zip(np.vstack([class_1, class_2]), np.array([1]*n_points + [-1]*n_points)):
    # Calcul de l'output du perceptron
    output = sign_activation(np.dot(weights, point) + bias)
    
    # Mise à jour des poids et du biais
    weights += learning_rate * (label - output) * point
    bias += learning_rate * (label - output)
```


**Question 5 :**. Tracez la ligne de séparation du perceptron sur le graphique des données.

``` python
# Coordonnées de la ligne de séparation
x_sep = np.linspace(-2, 2, 100)
y_sep = (-weights[0] * x_sep - bias) / weights[1]

# Affichage des points et de la ligne de séparation
plt.scatter(class_1[:, 0], class_1[:, 1], marker='x', color='blue', label='Class 1')
plt.scatter(class_2[:, 0], class_2[:, 1], marker='o', color='red', label='Class 2')
plt.plot(x_sep, y_sep, linestyle='--', color='green', label='Perceptron separator')

plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.legend()
```

### 2.2  vitesse de convergence et séparabilité


Dans cette question, nous allons étudier la vitesse de convergence du perceptron et observer comment il évolue en fonction du nombre d'itérations. La convergence dépend notamment de la séparabilité des données.


```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from matplotlib.animation import FuncAnimation
from IPython import display
```

**Question 1 :** Générez des données séparables et non séparables linéairement en utilisant la fonction `make_classification` de scikit-learn.

pour connaitre la forme de sont appel dans une cellue de code tapez :
```
make_classification?
```
pour ouvrir l'aide de la fonction, ou alors :   
```
make_classification??
```
pour ouvrir le code de la fonction.

Écrivez une fonction qui renvois un jeux de données
```python
def generate_data(separability = 1.0):
    X, y = make_classification(class_sep = separability ,n_samples=100, n_features=2, n_redundant=0,  n_informative=2, n_clusters_per_class=1, random_state=42)
    y[y == 0] = -1 # Remplacer les 0 par des -1 pour la classification binaire
    return  X, y
```


**Question 2 :** écrivez une fonction compute_perceptron qui calcul l'évolution de la frontière de séparation durant les itération :
```python
def compute_perceptron(X, y, alpha = 0.1, n_iter = 10):
    # Implémentation de l'algorithme du perceptron
    w = np.random.rand(3)
    errors = []
    w_history = [w.copy()]

    for t in range(n_iter):
        error_count = 0
        for x, target in zip(X, y):
            input_with_bias = np.append(x, 1)
            prediction = np.sign(np.dot(w, input_with_bias))
            update = alpha * (target - prediction) * input_with_bias
            w += update
            if target != prediction:
                error_count += 1
        w_history.append(w.copy())
        errors.append(error_count)
    return w_history, errors
```



**Question 3 :** Avec les lignes calculé on peut créer une annimation matplotlib avec la fonction suivante :

```python
def create_annimation(w_history, X, y):
    fig, ax = plt.subplots()
    n_iter = len(w_history)
    def animate(i):
        ax.clear()
        current_w = w_history[i]
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis')
        x_line = np.linspace(min(X[:, 0]), max(X[:, 0]), 100)
        y_line = (-current_w[0] * x_line - current_w[2]) / current_w[1]
        ax.plot(x_line, y_line, '-r')
        ax.set_title(f"Iteration: {i}")

    ani = FuncAnimation(fig, animate, frames=n_iter, interval=500)
    return ani
```


```python
X, y = generate_data(0.1)
w_history, errors =  compute_perceptron(X, y, alpha = 0.1, n_iter = 10)
test1 = create_annimation(w_history, X, y)
```

```python
test1.save("perceptron.gif")
```

pour sauvegarder l'annimation


pour afficher l' annimation :
```python
from IPython.display import Image
Image(url='perceptron.gif')
```

Version interactive pour explorer les paramètres :



```python
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Variables globales pour conserver le jeu de données et éviter de le régénérer inutilement
cached_data = None
cached_separability = None

# Définition de la fonction d'activation signe
def sign_activation(z):
    return 1 if z >= 0 else -1

# Fonction générant deux classes en 2D avec séparabilité contrôlée
def generate_linear_data(n_points=200, separability=6.0):
    """
    Génère deux classes de points en 2D.
    Les centres des distributions sont placés à -separability/2 et +separability/2,
    ce qui permet de contrôler la distance entre les deux classes.
    """
    center_1 = - separability / 2
    center_2 = + separability / 2
    class_1 = np.random.normal(loc=center_1, scale=1.0, size=(n_points, 2))
    class_2 = np.random.normal(loc=center_2, scale=1.0, size=(n_points, 2))
    return class_1, class_2

# Fonction exécutant le perceptron sur le jeu de données (en ne régénérant le dataset que si la séparabilité change)
def run_perceptron_interactif(n_points=500, separability=6.0, learning_rate=0.1, n_iter=10):
    global cached_data, cached_separability
    # Si les données n'ont jamais été générées ou si la séparabilité a changé, les générer et faire le split
    if cached_data is None or cached_separability != separability:
        # Génération du jeu complet (pour les deux classes)
        class_1, class_2 = generate_linear_data(n_points=n_points, separability=separability)
        X_full = np.vstack([class_1, class_2])
        y_full = np.array([1]*n_points + [-1]*n_points)
        # Mélanger les indices pour réaliser un split aléatoire
        indices = np.random.permutation(len(X_full))
        split_idx = int(0.8 * len(X_full))
        train_idx = indices[:split_idx]
        test_idx = indices[split_idx:]
        
        X_train = X_full[train_idx]
        y_train = y_full[train_idx]
        X_test = X_full[test_idx]
        y_test = y_full[test_idx]
        
        cached_data = (X_train, y_train, X_test, y_test)
        cached_separability = separability
    else:
        X_train, y_train, X_test, y_test = cached_data

    # Initialisation des poids et du biais du perceptron (apprentissage uniquement sur le jeu d'entraînement)
    weights = np.zeros(2)
    bias = 0

    # Phase d'apprentissage sur le jeu train, sur n_iter passages complets
    for _ in range(n_iter):
        for point, label in zip(X_train, y_train):
            prediction = sign_activation(np.dot(weights, point) + bias)
            update = learning_rate * (label - prediction)
            weights += update * point
            bias += update

    return X_train, y_train, X_test, y_test, weights, bias

# Fonction d'affichage de la frontière de décision et des points avec mise en forme spécifique
def afficher_frontiere(learning_rate, n_iter, separability):
    # Exécution de l'entraînement avec les paramètres choisis
    X_train, y_train, X_test, y_test, weights, bias = run_perceptron_interactif(
        learning_rate=learning_rate, n_iter=n_iter, separability=separability
    )
    
    # Prédiction sur les jeux d'entraînement et de test
    pred_train = np.array([sign_activation(np.dot(weights, x) + bias) for x in X_train])
    pred_test  = np.array([sign_activation(np.dot(weights, x) + bias) for x in X_test])
    
    # Calcul des accuracies
    train_acc = np.mean(pred_train == y_train)
    test_acc  = np.mean(pred_test == y_test)
    
    # Identification des indices mal classifiés par jeu et par classe
    idx_train_c1 = np.where((y_train == 1))[0]
    idx_train_c2 = np.where((y_train == -1))[0]
    idx_test_c1  = np.where((y_test == 1))[0]
    idx_test_c2  = np.where((y_test == -1))[0]
    
    idx_err_train_c1 = np.where((pred_train != y_train) & (y_train == 1))[0]
    idx_err_train_c2 = np.where((pred_train != y_train) & (y_train == -1))[0]
    idx_err_test_c1  = np.where((pred_test != y_test) & (y_test == 1))[0]
    idx_err_test_c2  = np.where((pred_test != y_test) & (y_test == -1))[0]
    
    plt.figure(figsize=(8, 6))
    
    # Affichage des données train avec des ronds pleins
    plt.scatter(X_train[idx_train_c1, 0], X_train[idx_train_c1, 1],
                c='blue', marker='o', label='Train Classe 1')
    plt.scatter(X_train[idx_train_c2, 0], X_train[idx_train_c2, 1],
                c='red', marker='o', label='Train Classe 2')
    
    # Affichage des données test avec des croix
    plt.scatter(X_test[idx_test_c1, 0], X_test[idx_test_c1, 1],
                c='blue', marker='x', label='Test Classe 1')
    plt.scatter(X_test[idx_test_c2, 0], X_test[idx_test_c2, 1],
                c='red', marker='x', label='Test Classe 2')
    
    # Surimpression des erreurs train : entourées d'un carré
    if idx_err_train_c1.size > 0:
        plt.scatter(X_train[idx_err_train_c1, 0], X_train[idx_err_train_c1, 1],
                    facecolors='none', edgecolors='blue', marker='s', s=150,
                    label='Erreur Train Classe 1')
    if idx_err_train_c2.size > 0:
        plt.scatter(X_train[idx_err_train_c2, 0], X_train[idx_err_train_c2, 1],
                    facecolors='none', edgecolors='red', marker='s', s=150,
                    label='Erreur Train Classe 2')
    
    # Surimpression des erreurs test : entourées d'un cercle
    if idx_err_test_c1.size > 0:
        plt.scatter(X_test[idx_err_test_c1, 0], X_test[idx_err_test_c1, 1],
                    facecolors='none', edgecolors='blue', marker='o', s=150,
                    label='Erreur Test Classe 1')
    if idx_err_test_c2.size > 0:
        plt.scatter(X_test[idx_err_test_c2, 0], X_test[idx_err_test_c2, 1],
                    facecolors='none', edgecolors='red', marker='o', s=150,
                    label='Erreur Test Classe 2')
    
    # Calcul et affichage de la frontière de décision
    x_min, x_max = plt.xlim()
    x_vals = np.linspace(x_min - 1, x_max + 1, 100)
    if weights[1] != 0:
        y_vals = (-weights[0] * x_vals - bias) / weights[1]
        plt.plot(x_vals, y_vals, 'g--', label='Frontière de décision')
    else:
        plt.axvline(x=-bias / weights[0], color='g', linestyle='--', label='Frontière de décision')
    
    plt.xlabel('Coordonnée X')
    plt.ylabel('Coordonnée Y')
    plt.title(f"Perceptron\nLR = {learning_rate}, Iterations = {n_iter}, Séparabilité = {separability}\n"
              f"Train Accuracy = {train_acc*100:.1f}%, Test Accuracy = {test_acc*100:.1f}%")
    plt.legend()
    plt.grid(True)
    plt.show()

# Création des sliders interactifs
lr_slider = widgets.FloatSlider(value=0.1, min=0.01, max=100.0, step=0.01, description='Learning Rate:')
iter_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Iterations:')
sep_slider = widgets.FloatSlider(value=3.0, min=0.1, max=5.0, step=0.1, description='Séparabilité:')

# Liaison de l'interactivité aux trois sliders
widgets.interact(afficher_frontiere, learning_rate=lr_slider, n_iter=iter_slider, separability=sep_slider)
```



### Ouverture

On peut essayer de faire varier les parametres de l'apprentissage : le alpha et le nombre d'iterations

On peut aussi regarder comment converge l'estimation en fonction de alpha et de la séparabilité des données

En dehors de l'intéret pédagogique une fois qu'on a compris comment fonctionne un perceptron on a tout intéret à utiliser des version déja implémenter.

Le code suivant reproduit l'expérience plus haute dans un cadre ou on peut changer la dimension de l'espace d'entré (plus seulement 2D) et fait une optimisation des hyperparamètre du perceptron (nombre d'itérations et pas de descente) :

```python
# grid search total epochs for the perceptron
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron

# defintion d'un jeux de données, ici on peut jouer sur la taille (n_features) et la spéparabilité des classes
X, y = make_classification(n_samples=1000, n_features=10, n_informative=10, class_sep=0.9 ,  n_redundant=0, random_state=1)

# definition d'un model de perceptron
model = Perceptron(eta0=0.0001)


# definition du mode d'évaluation par cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# definition de la grille d'exploration des paramètres
grid = dict()
# On chois
grid['max_iter'] = [1, 10, 100, 1000, 10000, 100000]
grid['eta0'] = [0.1, 0.01, 0.001, 0.0001]

# la recherche du parametrage optimal :
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X, y)

# le résultat
print('Mean Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)
# summarize all
means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']
for mean, param in zip(means, params):
    print(">%.3f with: %r" % (mean, param))
```
